In [ ]:
# #Instalação e Configuração
# %pip install yfinance==0.2.41
# %pip install crewai==0.28.8
# %pip install 'crewai[tools]'
# %pip install langchain==0.1.20
# %pip install langchain-openai==0.1.7
# %pip install langchain-community==0.0.38
# %pip install duckduckgo-search==5.3.0

In [ ]:
#Importações
import json
import os
import sys

import yfinance as yf

from datetime import datetime
from crewai import Agent, Task, Crew, Process
from langchain.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchResults

from api_settings import fetcher as fetch

In [ ]:
# Criação da Yahoo Finance Fetch Tool
def __fetchStock(ticket):
  return yf.download(ticket, start="2023-08-08", end="2024-08-08")

yf_fetch_tool = Tool(
  name = "Yahoo Finance Fetch Tool",
  description = "Fetches stock price history for {ticket} from the Yahoo Finance API",
  func = lambda ticket: __fetchStock(ticket)
)

# def response(ticket):
#   return yf_fetch_tool.run(ticket)

In [ ]:
# Importando OPENAI LLM - GPT
os.environ['OPENAI_API_KEY'] = fetch.fetchKey('api_settings/openAIKey.txt', 'openAIKey')
llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
AGENT_trendsAnalyst = Agent(
  role = "Senior Market Trends Analyst",
  goal = "Find the {ticket} stock price and analyze market trends",
  backstory = """You're a highly experienced investor and you're an expert in analyzing market trends 
  and make reasonably accurate predictions about future stock price growth""",
  verbose = True,
  llm = llm,
  max_iter = 5,
  memory = True,
  allow_delegation = False,
  tools = [yf_fetch_tool]
)

In [ ]:
TASK_getStockPrice = Task(
  description = "Analyze the stock {ticket} price history and create a trend analysis of up, down or sideways",
  expected_output = """ Specify the current trend - up, down or sideways.
  eg. stock='AAPL, price UP'""",
  agent = AGENT_trendsAnalyst
)

In [ ]:
search_tool = DuckDuckGoSearchResults(backend='news', num_results=10)

In [ ]:
AGENT_newsAnalyst = Agent(
  role = "Senior Market News Analyst",
  goal = """Create a short summary of news regarding the stock {ticket} 
  and specify the current trend based on the context of the news - up, down or sideways. 
  For each requested stock, specify a number between 0 and 100, where 0 indicates extreme fear and 100 indicates extreme greed.""",
  backstory = """You're a senior market analyst with over a decade in experience, you specialize in predicting future trends based on current news.
  You're also a master level analyst in the traditional markets and you've got a deep understanding of human psychology and market trends.
  You're capable of reading, interpreting and understanding current news - their headlines and contents -  however, you still regard them with a healthy dose of skepticism, you also prefer to read articles
  from trustworthy sources""",
  verbose = True,
  llm = llm,
  max_iter = 10,
  memory = True,
  allow_delegation = False,
  tools = [search_tool]
)

In [ ]:
TASK_getNews = Task(
  description = f"""Take the stock and always include BTC and ETH to it (if not requested).
  Use the search tool to search each one individually
  
  The current date is {datetime.now()}
  
  Compose the results into a helpful report""",
  expected_output = """ A summary of the overall market and a single-sentenced summary for each requested asset. Include a fear/greed score for each asset based on the current news.
  Use the following format:
  <ASSET NAME>
  <SUMMARY BASED ON NEWS>
  <TREND PREDICTION>
  <FEAR/GREED INDEX>""",
  agent = AGENT_newsAnalyst
)

In [ ]:
AGENT_writerAnalyst = Agent(
  role = "Senior Stock Analyst",
  goal = "Write an insightful, informative and compelling 3 paragraph-long newsletter report based on stock price trends and news headline trends.",
  backstory = """You're a well-regarded and well-known market analyst, your newsletter is also widely considered to be trustworthy.
  You're able to understand the intricate and complex market behaviors and you're capable to create compelling stories and narratives that resonate with the wider audiences 
  that might not necessarily understand the market as well as you do.
  
  You understand macrofactors and combine multiple theories - eg. cycle theory and fundamental analysis.
  You're capable of holding multiple opinions when analyzing anything""",
  verbose = True,
  llm = llm,
  max_iter = 5,
  memory = True,
  allow_delegation = True,
)

In [ ]:
TASK_writeAnalysis = Task(
  description = """Use the Stock Price Trends and Market News Trends analysis reports to create
  and write a brief newsletter analyzing the {ticket} company featuring the most relevant and important points.
  Focus on the stock price trends, news and the fear/greed index score. What are the near future considerations?
  Also include the previous analysis of the stock trend and news summary.""",
  expected_output = """ An eloquent 3 paragraph newsletter formatted as markdown in an easily readable manner. It should contain:
  - 3 Bullets Executive Summary
  - Introduction - Set the overall picture and spike up the interest of the reader
  - Main Part that provides the actual bulk of the analysis including the summary of the news and the fear/greed index.
  - Summary - key facts and concrete future trends predictions - up, down or sideways.
  """,
  agent = AGENT_writerAnalyst,
  context = [TASK_getStockPrice, TASK_getNews]
)

In [ ]:
crew = Crew(
  agents = [AGENT_trendsAnalyst, AGENT_newsAnalyst, AGENT_writerAnalyst],
  tasks = [TASK_getStockPrice, TASK_getNews, TASK_writeAnalysis],
  verbose = 2,
  process = Process.hierarchical,
  full_output = True,
  share_crew = False,
  manager_llm = llm,
  max_iter = 15
)

In [ ]:
results = crew.kickoff(inputs={'ticket': 'AAPL'})